In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
### LAR

def cleanLAR(year):
    fp = f"C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\LAR\\{year}.txt"

    # Create dictionary of columns where 'Exempt' is a null value
    exempts = ['combined_loan_to_value_ratio', 'interest_rate', 'rate_spread', 
    'total_loan_costs', 'total_points_and_fees', 'origination_charges',
    'discount_points', 'lender_credits', 'loan_term',
    'prepayment_penalty_term', 'intro_rate_period', 'property_value',
    'total_units', 'multifamily_affordable_units', 'debt_to_income_ratio']
    exemptNulls = dict(zip(exempts, ['Exempt']*len(exempts)))

    return pd.read_csv(fp, delimiter='|',  
                 na_values = exemptNulls,
                 dtype={
                    'state_code':'str', 
                    'applicant_age':'str', 
                    'co_applicant_age':'str'
                }
        )


In [3]:
lar = cleanLAR(2022)

with sql.connect("C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\HMDA.sqlite") as conn:
    lar.to_sql('LAR', conn, index=False)

for year in list(range(2018, 2022)):
    print(year)
    del lar
    lar = cleanLAR(year)
    with sql.connect("C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\HMDA.sqlite") as conn:
        lar.to_sql('LAR', conn, index=False, if_exists='append')


C:\Users\WilliamRobinson\AppData\Local\Temp\ipykernel_6720\506808745.py:14: DtypeWarning: Columns (43,46,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fp, delimiter='|',


2018


C:\Users\WilliamRobinson\AppData\Local\Temp\ipykernel_6720\506808745.py:14: DtypeWarning: Columns (4,5,43,46,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fp, delimiter='|',


2019


C:\Users\WilliamRobinson\AppData\Local\Temp\ipykernel_6720\506808745.py:14: DtypeWarning: Columns (5,43,46,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fp, delimiter='|',


2020


C:\Users\WilliamRobinson\AppData\Local\Temp\ipykernel_6720\506808745.py:14: DtypeWarning: Columns (5,43,46,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fp, delimiter='|',


2021


C:\Users\WilliamRobinson\AppData\Local\Temp\ipykernel_6720\506808745.py:14: DtypeWarning: Columns (5,43,46,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fp, delimiter='|',


In [4]:
del lar 

In [5]:
### Entities

lei = pd.DataFrame()

def cleanLEI(year, df):
    fp = f"C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\Entities\\{year}.txt"
    
    temp = pd.read_csv(fp, delimiter='|')
    temp['name_state'] = [f'{x} ({y})' for x, y in zip(temp.respondent_name, temp.respondent_state)]
    
    return pd.concat(
        [df, temp],
        ignore_index=True
    )
    

In [6]:
### Loan Limits

ll = pd.DataFrame()

def cleanLL(year, df):
    fp = f"C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\LoanLimits\\{year}.xlsx"
    
    cols = ['state_code', 'county_code', 'county_name', 'state', 'cbsa', 'one_unit_limit',
        'two_unit_limit', 'three_unit_limit', 'four_unit_limit']

    temp = pd.read_excel(fp, skiprows=1, sheet_name='GSE Limits', names=cols)
    temp['year'] = year

    return pd.concat(
        [df, temp], 
        ignore_index=True
    )

In [7]:
for year in list(range(2018, 2023)):
    lei = cleanLEI(year, lei)

In [8]:
for year in list(range(2018, 2023)):
    ll = cleanLL(year, ll)

In [9]:
with sql.connect("C:\\Users\\WilliamRobinson\\OneDrive - HW Publishing LLC\\Documents\\Data\\HMDA\\HMDA.sqlite") as conn:
    lei.to_sql('Entities', conn, index=False)
    ll.to_sql('LoanLimits', conn, index=False)

<a href="https://www.fhfa.gov/DataTools/Downloads/Pages/Conforming-Loan-Limit.aspx">Conforming loan limits</a> by year